In [6]:
# Load necessary modules here
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
from torch.autograd import Variable
import os


class Bottleneck(nn.Module):
    '''
        the above mentioned bottleneck, including two conv layer, one's kernel size is 1×1, another's is 3×3

        after non-linear operation, concatenate the input to the output
    '''
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        
        # input and output are concatenated here
        out = torch.cat([out,x], 1)
        return out


class Transition(nn.Module):
    '''
        transition layer is used for down sampling the feature
        
        when compress rate is 0.5, out_planes is a half of in_planes
    '''
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        
        out = self.conv(F.relu(self.bn(x)))
        # use average pooling change the size of feature map here
        out = F.avg_pool2d(out, 2)
        return out 

    
class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=5):
        super(DenseNet, self).__init__()
        '''
        Args:
            block: bottleneck
            nblock: a list, the elements is number of bottleneck in each denseblock
            growth_rate: channel size of bottleneck's output
            reduction: 
        '''
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)
        
        # a DenseBlock and a transition layer
        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        # the channel size is superposed, mutiply by reduction to cut it down here, the reduction is also known as compress rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes
        
        # a DenseBlock and a transition layer
        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        # the channel size is superposed, mutiply by reduction to cut it down here, the reduction is also known as compress rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        # a DenseBlock and a transition layer
        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        # the channel size is superposed, mutiply by reduction to cut it down here, the reduction is also known as compress rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        # only one DenseBlock 
        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        # the last part is a linear layer as a classifier
        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(2727, num_classes)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        
        # number of non-linear transformations in one DenseBlock depends on the parameter you set
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [7]:
def densenet():
    return DenseNet(Bottleneck, [12, 14, 12, 14])

In [8]:
import pandas as pd
import cv2
from PIL import Image
import torchvision.transforms as transforms

In [9]:
class Mydataset(Dataset):
    
    def __init__(self, csv_path, img_path):
        self.datainfo = pd.read_csv(csv_path)
        self.img = np.asarray(self.datainfo.iloc[:, 0])
        self.label = np.asarray(self.datainfo.iloc[:, 1])
        self.length = len(self.datainfo)
        self.trainpath = img_path
        self.transform = transforms.Compose([
        transforms.Resize(100),
        transforms.RandomCrop(100, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # Normalize a tensor image with mean and standard variance
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
        print(self.length)
        print(self.datainfo.dtypes)
        
    def __getitem__(self, index):
        imgpath = self.trainpath + self.img[index] + '.png'
        image = Image.open(imgpath).convert('RGB')
        imagetensor = self.transform(image)
        imagelabel = self.label[index]
        return (imagetensor, imagelabel)

    def __len__(self):
        return self.length

In [10]:
mydataset = Mydataset('./train.csv', './train_images/')

3662
id_code      object
diagnosis     int64
dtype: object


In [10]:
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable


def train(epoch, model, lossFunction, optimizer, device, trainloader):
    """train model using loss_fn and optimizer. When this function is called, model trains for one epoch.
    Args:
        train_loader: train data
        model: prediction model
        loss_fn: loss function to judge the distance between target and outputs
        optimizer: optimize the loss function
        get_grad: True, False
    output:
        total_loss: loss
        average_grad2: average grad for hidden 2 in this epoch
        average_grad3: average grad for hidden 3 in this epoch
    """
    print('\nEpoch: %d' % epoch)
    model.train()     # enter train mode
    train_loss = 0    # accumulate every batch loss in a epoch
    correct = 0       # count when model' prediction is correct i train set
    total = 0         # total number of prediction in train set
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device) # load data to gpu device
        inputs, targets = Variable(inputs), Variable(targets)
        optimizer.zero_grad()            # clear gradients of all optimized torch.Tensors'
        outputs = model(inputs)          # forward propagation return the value of softmax function
        loss = lossFunction(outputs, targets) #compute loss
        loss.backward()                  # compute gradient of loss over parameters 
        optimizer.step()                 # update parameters with gradient descent 

        train_loss += loss.item()        # accumulate every batch loss in a epoch
        _, predicted = outputs.max(1)    # make prediction according to the outputs
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item() # count how many predictions is correct
        
        if (batch_idx+1) % 100 == 0:
            # print loss and acc
            print( 'Train loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print( 'Train loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    torch.save(model, './densenet.pt')
    
    
def test(model, lossFunction, optimizer, device, testloader):
    """
    test model's prediction performance on loader.  
    When thid function is called, model is evaluated.
    Args:
        loader: data for evaluation
        model: prediction model
        loss_fn: loss function to judge the distance between target and outputs
    output:
        total_loss
        accuracy
    """
    global best_acc
    model.eval() #enter test mode
    test_loss = 0 # accumulate every batch loss in a epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = lossFunction(outputs, targets) #compute loss

            test_loss += loss.item() # accumulate every batch loss in a epoch
            _, predicted = outputs.max(1) # make prediction according to the outputs
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() # count how many predictions is correct
        # print loss and acc
        print('Test Loss: %.3f  | Test Acc: %.3f%% (%d/%d)'
            % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

        
def data_loader():
    # define method of preprocessing data for evaluating
    
    # prepare dataset by ImageFolder, data should be classified by directory
    trainset = mydataset

    # Data loader. 

    # Combines a dataset and a sampler, 

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

    
    return trainloader

def run(model, num_epochs):
    
    # load model into GPU device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    if device == 'cuda':
        model = torch.nn.DataParallel(model)
        cudnn.benchmark = True

    # define the loss function and optimizer

    lossFunction = nn.CrossEntropyLoss()
    lr = 0.00390625
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    trainloader = data_loader()
    for epoch in range(num_epochs):
        train(epoch, model, lossFunction, optimizer, device, trainloader)
        # test(model, lossFunction, optimizer, device, testloader)
        if (epoch + 1) % 30 == 0 :
            lr = lr / 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

In [12]:
# start training and testing
model = densenet()
# num_epochs is adjustable
run(model, num_epochs=20)


Epoch: 0
Train loss: 1.142 | Train Acc: 58.312% (933/1600)
Train loss: 1.047 | Train Acc: 61.344% (1963/3200)
Train loss: 1.029 | Train Acc: 62.206% (2278/3662)


C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:251: UserWarning: Couldn't retrieve source code for container of type DenseNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Bottleneck. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Transition. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



Epoch: 1
Train loss: 0.899 | Train Acc: 69.250% (1108/1600)
Train loss: 0.894 | Train Acc: 68.719% (2199/3200)
Train loss: 0.897 | Train Acc: 68.651% (2514/3662)

Epoch: 2
Train loss: 0.882 | Train Acc: 67.750% (1084/1600)
Train loss: 0.881 | Train Acc: 68.125% (2180/3200)
Train loss: 0.888 | Train Acc: 68.159% (2496/3662)

Epoch: 3
Train loss: 0.837 | Train Acc: 70.375% (1126/1600)
Train loss: 0.851 | Train Acc: 69.719% (2231/3200)
Train loss: 0.847 | Train Acc: 70.126% (2568/3662)

Epoch: 4
Train loss: 0.854 | Train Acc: 69.562% (1113/1600)
Train loss: 0.828 | Train Acc: 70.812% (2266/3200)
Train loss: 0.825 | Train Acc: 70.781% (2592/3662)

Epoch: 5
Train loss: 0.813 | Train Acc: 70.125% (1122/1600)
Train loss: 0.826 | Train Acc: 70.375% (2252/3200)
Train loss: 0.822 | Train Acc: 70.781% (2592/3662)

Epoch: 6
Train loss: 0.839 | Train Acc: 69.750% (1116/1600)
Train loss: 0.796 | Train Acc: 71.188% (2278/3200)
Train loss: 0.803 | Train Acc: 70.890% (2596/3662)

Epoch: 7
Train loss: 

In [14]:
run(model, num_epochs=60)


Epoch: 0
Train loss: 0.716 | Train Acc: 74.250% (1188/1600)
Train loss: 0.725 | Train Acc: 73.875% (2364/3200)
Train loss: 0.726 | Train Acc: 73.539% (2693/3662)

Epoch: 1
Train loss: 0.717 | Train Acc: 73.625% (1178/1600)
Train loss: 0.723 | Train Acc: 72.531% (2321/3200)
Train loss: 0.721 | Train Acc: 73.020% (2674/3662)

Epoch: 2
Train loss: 0.734 | Train Acc: 71.812% (1149/1600)
Train loss: 0.722 | Train Acc: 72.750% (2328/3200)
Train loss: 0.713 | Train Acc: 73.129% (2678/3662)

Epoch: 3
Train loss: 0.704 | Train Acc: 73.812% (1181/1600)
Train loss: 0.717 | Train Acc: 73.188% (2342/3200)
Train loss: 0.713 | Train Acc: 73.211% (2681/3662)

Epoch: 4
Train loss: 0.728 | Train Acc: 73.250% (1172/1600)
Train loss: 0.720 | Train Acc: 73.500% (2352/3200)
Train loss: 0.715 | Train Acc: 73.758% (2701/3662)

Epoch: 5
Train loss: 0.717 | Train Acc: 72.625% (1162/1600)
Train loss: 0.716 | Train Acc: 73.062% (2338/3200)
Train loss: 0.710 | Train Acc: 73.321% (2685/3662)

Epoch: 6
Train loss: 

In [11]:
mytestset = Mydataset('./sample_submission.csv', './test_images/')

1928
id_code      object
diagnosis     int64
dtype: object


In [12]:
testloader = torch.utils.data.DataLoader(mytestset, batch_size=1, shuffle=False)
result = []

In [13]:
def test(model, device, testloader):
    model.eval() #enter test mode
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1) # make prediction according to the outputs
            result.append(predicted.cpu().numpy())

In [14]:
test(model, 'cuda', testloader)

In [44]:
print(len(result))
submission = pd.read_csv('./sample_submission.csv')
im = submission.iloc[:, 0]
print(len(im))

1928
1928


In [49]:
result = np.array(result).flatten()
print(result)

[2 2 2 ... 2 4 0]


In [50]:

df = pd.DataFrame({'id_code':im,'diagnosis':result})
df.to_csv('submission.csv',index=False)

In [51]:
print(submission.dtypes)
print(df.dtypes)

id_code      object
diagnosis     int64
dtype: object
id_code      object
diagnosis     int64
dtype: object


In [7]:
model = torch.load('./densenet.pt')

In [3]:
import torch
print(torch.__version__)


1.0.1


In [15]:
torch.save(model.state_dict(),"./densenet.pth")